## Transpose Table

In [ ]:
import csv

# tab-delimited table
filename = '.tables/.esv-2016-changes.tsv'

md_table = '|Bible Reference|2011 Text|2016 Text|\n|---|---|---|\n'

# open the file in read mode
with open(filename, 'r', newline='', encoding='utf-8') as tsvfile:
    # Create a csv reader object, specifying the tab delimiter
    tsv_reader = csv.reader(tsvfile, delimiter='\t')

    # combine every other row
    for row in tsv_reader:
        if len(row) == 1:
            ref = row[0]
        elif len(row) == 2:
            md_table += f'|{ref}|{row[1]}|{row[0]}|\n'

# save Markdown file
with open('.tables/.esv-2016-changes.md', 'w') as f:
    print(md_table, file=f)

In [ ]:
# from IPython.display import Markdown
# Markdown(md_table)

## Mark Changes

In [15]:
import pandas as pd
import csv
import difflib

# load table from file
revisions = pd.read_csv('.tables/.esv-2016-changes.md', 
    sep='|', skipinitialspace=True, usecols=[1,2,3], 
    skiprows=[1], quoting=csv.QUOTE_NONE)

In [16]:
# create markdown table
md_table = '|Bible Reference|2011 Text|2016 Text|\n|---|---|---|\n'

# create sequence matcher
s = difflib.SequenceMatcher()

# iterate through changes
for index, row in revisions.iterrows():
    ref = row['Bible Reference']
    text1 = row['2011 Text']
    text2 = row['2016 Text']
    changes1 = ''
    changes2 = ''

    # Get opcodes for word-level comparison
    s.set_seqs(text1.split(), text2.split())
    opcodes = s.get_opcodes()

    # format changes
    for tag, i1, i2, j1, j2 in opcodes:
        if tag == 'equal':
            # add unchanged words
            changes1 += f' {" ".join(text1.split()[i1:i2])}'
            changes2 += f' {" ".join(text2.split()[j1:j2])}'
        elif tag == 'replace':
            # Highlight changed words
            changes1 += f' **{" ".join(text1.split()[i1:i2])}**'
            changes2 += f' **{" ".join(text2.split()[j1:j2])}**'
        elif tag == 'delete':
            changes1 += f' **{" ".join(text1.split()[i1:i2])}**'
            # changes2 += f' **{" ".join(text2.split()[j1:j2])}**'
        elif tag == 'insert':
            # changes1 += f' **{" ".join(text1.split()[i1:i2])}**'
            changes2 += f' **{" ".join(text2.split()[j1:j2])}**'

    md_table += f'|{ref}|{changes1}|{changes2}|\n'

# save to file
with open('.tables/.esv-2016-changes-formatted.md', 'w') as f:
    print(md_table, file=f)

# Markdown(md_table)